In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import TimeSeriesSplit
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
sbmsn = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
print(f"The shape of training data is {train.shape}")
print(f"The shape of test data is {test.shape}")

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'])
test['date_time'] = pd.to_datetime(test['date_time'])

In [ ]:
train['weekday'] = train['date_time'].dt.weekday
test['weekday'] = test['date_time'].dt.weekday
train['dayofmonth'] = train['date_time'].dt.day
test['dayofmonth'] = test['date_time'].dt.day
train['quarter'] = train['date_time'].dt.quarter
test['quarter'] = test['date_time'].dt.quarter

In [ ]:
print(f"The starting date in train data in {train['date_time'].min()}")
print(f"The ending date in train data in {train['date_time'].max()}")

In [ ]:
print(f"The starting date in test data in {test['date_time'].min()}")
print(f"The ending date in test data in {test['date_time'].max()}")

In [ ]:

def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))


In [ ]:
# Features to use:
columns_to_use = ['quarter','weekday','dayofmonth','deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']

In [ ]:
# tscv = TimeSeriesSplit(n_splits=3)
# for train_index, val_index in tscv.split(train[columns_to_use]):
#     for target in ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']:
#         X_train, y_train = train.loc[train_index, columns_to_use], train.loc[train_index, target]
#         X_val, y_val = train.loc[val_index, columns_to_use], train.loc[val_index,target]
#         model = LGBMRegressor()
#         model.fit(X_train, y_train)
#         val_preds = model.predict(X_val)
#         print(f"The RMSLE is {rmsle(y_val, val_preds)}")
#         preds = model.predict(test[columns_to_use])
#         sbmsn[target] += preds/3

In [ ]:
for target in ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']:
    model = LGBMRegressor()
    model.fit(train[columns_to_use], train[target])
    preds = model.predict(test[columns_to_use])
    sbmsn[target] = preds

In [ ]:
sbmsn.to_csv('submission.csv', index=False)